# Comet + R with nnet

This notebook is based on:

* https://www.rdocumentation.org/packages/nnet/versions/7.3-13/topics/nnet

To get started with Comet and R, please see:
https://www.comet.ml/docs/r-sdk/getting-started/

Specifically, you need to create a .comet.yml file
or add your Comet API key to create_experiment()

```r
install.packages("cometr")
```

First we identify which libraries we wish to use:

In [1]:
library(cometr)
library(nnet)
library(stringr)

Next, we create a Comet experiment marking what we would like to log to the server:

In [2]:
exp <- create_experiment(
  keep_active = TRUE,
  log_output = FALSE,
  log_error = FALSE,
  log_code = TRUE,
  log_system_details = TRUE,
  log_git_info = TRUE
)

Experiment created: https://www.comet.ml/dsblank/cometr/7e1ea5611977445e9c9041b8eb293714 


Note: the notebook source isn't logged through the experiment. 

In [3]:
exp$add_tags(c("made with nnet"))

In [4]:
# sample the iris data:

sample_size <- 25 # of each iris type
total_size <- 50
total_size2 <- total_size * 2

In [5]:
exp$log_parameter("sample_size", sample_size)

In [6]:
ir <- rbind(iris3[,,1], iris3[,,2], iris3[,,3])
targets <- class.ind(c(
  rep("s", total_size),
  rep("c", total_size),
  rep("v", total_size))
)
samp <- c(
  sample(1:total_size, sample_size),
  sample((total_size + 1):(total_size * 2), sample_size),
  sample(((total_size * 2) + 1):(total_size * 3), sample_size)
)

weight_decay <- 5e-4
epochs <- 200
hidden_layer_size <- 2
initial_random_weight_range <- 0.1

In [7]:
exp$log_parameter("weight_decay", weight_decay)
exp$log_parameter("epochs", epochs)
exp$log_parameter("hidden_layer_size", hidden_layer_size)
exp$log_parameter("initial_random_weight_range", initial_random_weight_range)

Ideally, all we need to do next is:

```r
nnet(
    ir[samp,],
    targets[samp,],
    size = hidden_layer_size,
    rang = initial_random_weight_range,
    decay = weight_decay,
    maxit = epochs
)
```

However, we wish to log the "loss" values from the training. Unfortunately, the loop that does the processing is in C. But we can grab the output, parse it, and then log it.

In [8]:
ir1 <- NULL

train <- function() {
  ir1 <<- nnet(
    ir[samp,],
    targets[samp,],
    size = hidden_layer_size,
    rang = initial_random_weight_range,
    decay = weight_decay,
    maxit = epochs)
    ir1
}

output <- capture.output(train(), split = TRUE)
output <- strsplit(output, "\n")

# "initial  value 57.703088 "
for (match in str_match(output, "^initial\\s+value\\s+([-+]?[0-9]*\\.?[0-9]+)")[,2]) {
  if (!is.na(match)) {
     exp$log_metric("loss", match, step=0)
  }
}

# "iter  10 value 46.803951"
matrix = str_match(output, "^iter\\s+(\\d+)\\s+value\\s+([-+]?[0-9]*\\.?[0-9]+)")
for (i in 1:nrow(matrix)) {
  match = matrix[i,]
  if (!is.na(match[2])) {
     exp$log_metric("loss", match[3], step=match[2])
  }
}

# weights:  19
initial  value 55.315165 
iter  10 value 34.877822
iter  20 value 0.745746
iter  30 value 0.458811
iter  40 value 0.420603
iter  50 value 0.398487
iter  60 value 0.392074
iter  70 value 0.387933
iter  80 value 0.384593
iter  90 value 0.383557
iter 100 value 0.383034
iter 110 value 0.382928
iter 120 value 0.382837
iter 130 value 0.382762
iter 140 value 0.382754
iter 150 value 0.382749
iter 160 value 0.382748
final  value 0.382747 
converged
a 4-2-3 network with 19 weights
options were - decay=5e-04


Now, we'll test the trained model by creating a confusion matrix:

In [9]:
test.cl <- function(true, pred) {
    true <- max.col(true)
    cres <- max.col(pred)
    table(true, cres)
}
test.cl(targets[-samp,], predict(ir1, ir[-samp,]))

    cres
true  1  2  3
   1 22  0  3
   2  0 25  0
   3  0  0 25

In [10]:
exp$log_html("
<h1>Comet nnet Example</h1>

<p>This example demonstrates using the nnet library on the iris dataset.</p>

<p>See the Output tab for confusion matrix.</p>

<ul>
<li><a href=https://github.com/comet-ml/cometr/blob/master/inst/train-examples/nnet-example.R>github.com/comet-ml/cometr/inst/train-example/nnet-example.R</a></li>
</ul>

<p>For help on the Comet R SDK, please see: <a href=https://www.comet.ml/docs/r-sdk/getting-started/>www.comet.ml/docs/r-sdk/getting-started/</a></p>
")

In [11]:
exp$log_other(key = "Created by", value = "cometr")

At this point, save your notebook. We'll then upload it as an asset:

In [12]:
exp$upload_asset(
  "nnet-example.ipynb",
  type = "notebook",
)

In [13]:
exp$print()
exp$stop()

Comet experiment https://www.comet.ml/dsblank/cometr/7e1ea5611977445e9c9041b8eb293714 
